# 🔧 TRANSMUTATION CONTROL — Debug & Testing

```
╔══════════════════════════════════════════════════════════════════════════════╗
║                 FRÉGATE 01_TRANSMUTATION — CONTROL PANEL                     ║
║                    Debug • Test EMOCA • Inspect • Preview                    ║
╚══════════════════════════════════════════════════════════════════════════════╝
```

**Objectif**: Tester individuellement chaque composant du pipeline TRANSMUTATION.

---

## 📦 Setup & Mount Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Configuration
DRIVE_ROOT = "/content/drive/MyDrive/DRIVE_EXODUS_V2"
UNIT_ROOT = f"{DRIVE_ROOT}/01_ANIMATION_ENGINE"
CODEBASE = f"{UNIT_ROOT}/CODEBASE"

# Add codebase to path
import sys
sys.path.insert(0, CODEBASE)

print(f"✓ Drive monté")
print(f"✓ Unit Root: {UNIT_ROOT}")

In [ ]:
# Install dependencies
!pip install -q torch torchvision opencv-python-headless scipy omegaconf Pillow tqdm

## 🔍 Test 1: Vérification des Chemins

In [ ]:
from pathlib import Path

paths_to_check = [
    ("Unit Root", UNIT_ROOT),
    ("Codebase", CODEBASE),
    ("IN_INPUTS", f"{UNIT_ROOT}/IN_INPUTS"),
    ("OUT_BAKED", f"{UNIT_ROOT}/OUT_BAKED"),
    ("Body Motions", f"{UNIT_ROOT}/IN_INPUTS/body_motions"),
    ("Source Videos", f"{UNIT_ROOT}/IN_INPUTS/source_videos"),
    ("Actor Models", f"{UNIT_ROOT}/IN_INPUTS/actor_models"),
]

print("=== Vérification Chemins ===")
for name, path in paths_to_check:
    exists = Path(path).exists()
    status = "✓" if exists else "✗"
    print(f"{status} {name}: {path}")

In [ ]:
# Lister les fichiers disponibles
import os

print("\n=== Fichiers Inputs ===")

for folder in ["body_motions", "source_videos", "actor_models"]:
    folder_path = f"{UNIT_ROOT}/IN_INPUTS/{folder}"
    print(f"\n📁 {folder}:")
    if Path(folder_path).exists():
        files = [f for f in os.listdir(folder_path) if not f.startswith('.')]
        if files:
            for f in files:
                print(f"   - {f}")
        else:
            print("   (vide)")

## 🎭 Test 2: EMOCA Extraction

In [ ]:
# Test Facial Extractor
from facial_extractor import EMOCAExtractor, ARKIT_52_BLENDSHAPES

print(f"ARKit Blendshapes supportés: {len(ARKIT_52_BLENDSHAPES)}")
print("\nListe complète:")
for i, name in enumerate(ARKIT_52_BLENDSHAPES):
    print(f"  {i+1:2d}. {name}")

In [ ]:
# Test extraction sur une vidéo (si disponible)
TEST_VIDEO = f"{UNIT_ROOT}/IN_INPUTS/source_videos/test.mp4"  # Modifier selon fichier disponible
EMOCA_PATH = f"{DRIVE_ROOT}/EXODUS_AI_MODELS/emoca"

if Path(TEST_VIDEO).exists():
    print(f"Test vidéo trouvée: {TEST_VIDEO}")
    
    extractor = EMOCAExtractor(EMOCA_PATH)
    
    # Extraire seulement 30 frames pour le test
    face_data = extractor.extract_arkit_from_video(TEST_VIDEO, end_frame=30)
    
    print(f"\nRésultat:")
    print(f"  FPS: {face_data['fps']}")
    print(f"  Frames: {len(face_data['frames'])}")
    
    if face_data['frames']:
        sample = face_data['frames'][0]
        print(f"\nÉchantillon Frame 0:")
        for key, val in list(sample['blendshapes'].items())[:10]:
            print(f"    {key}: {val:.4f}")
else:
    print(f"Aucune vidéo test trouvée à: {TEST_VIDEO}")
    print("Placez une vidéo .mp4 dans IN_INPUTS/source_videos/")

## 🔄 Test 3: Synchronisation

In [ ]:
from sync_engine import SyncEngine

sync = SyncEngine(verbose=True)

# Test méthode marqueur
print("=== Test Sync Engine ===")

# Exemple: dans la vidéo, le clap est à la frame 150
#          dans le FBX, le mouvement commence à la frame 100
offset = sync.calculate_offset(
    method="marker",
    marker_video=150,
    marker_fbx=100
)

print(f"\nOffset calculé: {offset} frames")
print("→ Signifie: la vidéo est en avance de 50 frames sur le FBX")

In [ ]:
# Validation sync
is_valid, msg = sync.validate_sync(
    body_length=1000,  # FBX a 1000 frames
    face_length=900,   # Vidéo a 900 frames
    offset=50
)

print(f"Validation: {msg}")

start, end = sync.get_frame_range(1000, 900, 50)
print(f"Plage effective: frames {start} - {end}")

## 📈 Test 4: Smoothing

In [ ]:
import numpy as np
from smoothing import savgol_smooth, adaptive_smooth, compute_smoothing_metrics

# Générer signal test avec bruit
np.random.seed(42)
n_frames = 100
t = np.linspace(0, 2 * np.pi, n_frames)

# Signal: smile qui monte puis descend
clean_signal = np.sin(t) * 0.4 + 0.5
noisy_signal = clean_signal + np.random.normal(0, 0.08, n_frames)
noisy_signal = np.clip(noisy_signal, 0, 1)

# Appliquer lissage
smoothed = savgol_smooth(noisy_signal, window=7)

print("=== Test Smoothing ===")
print(f"Signal original: mean={np.mean(noisy_signal):.3f}, std={np.std(noisy_signal):.3f}")
print(f"Signal lissé: mean={np.mean(smoothed):.3f}, std={np.std(smoothed):.3f}")

In [ ]:
# Visualisation
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 4))
plt.plot(noisy_signal, alpha=0.5, label='Brut (avec bruit)')
plt.plot(smoothed, linewidth=2, label='Lissé (Savitzky-Golay)')
plt.plot(clean_signal, '--', alpha=0.7, label='Signal original')
plt.xlabel('Frame')
plt.ylabel('Blendshape Value')
plt.title('Comparaison Smoothing')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 🚀 Test 5: Dry Run Pipeline Complet

In [ ]:
# Test dry-run du script principal
!python {CODEBASE}/EXO_01_TRANSMUTATION.py --help

In [ ]:
# Exemple de dry-run (validation chemins sans exécution)
# Décommenter et adapter les noms de fichiers

# !python {CODEBASE}/EXO_01_TRANSMUTATION.py \
#     --drive-root {DRIVE_ROOT} \
#     --body-fbx motion.fbx \
#     --video source.mp4 \
#     --actor-blend avatar.blend \
#     --dry-run -v

## 📊 Inspection Face Data JSON

In [ ]:
import json

def inspect_face_json(json_path):
    """Inspecte un fichier JSON de données faciales."""
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    print(f"=== Inspection: {json_path} ===")
    print(f"FPS: {data.get('fps', 'N/A')}")
    print(f"Total Frames: {data.get('total_frames', len(data.get('frames', [])))}")
    print(f"Frames extraites: {len(data.get('frames', []))}")
    
    if data.get('frames'):
        # Stats par blendshape
        import numpy as np
        
        blendshape_names = list(data['frames'][0]['blendshapes'].keys())
        
        print(f"\nBlendshapes: {len(blendshape_names)}")
        print("\nTop 10 blendshapes actifs (par max):")
        
        max_values = {}
        for name in blendshape_names:
            values = [f['blendshapes'].get(name, 0) for f in data['frames']]
            max_values[name] = max(values)
        
        sorted_bs = sorted(max_values.items(), key=lambda x: x[1], reverse=True)[:10]
        for name, val in sorted_bs:
            print(f"  {name}: {val:.3f}")

# Exemple d'utilisation (décommenter si fichier existe)
# inspect_face_json(f"{UNIT_ROOT}/OUT_BAKED/test_face.json")

---
## ✅ Checklist Debug

- [ ] Chemins vérifiés
- [ ] EMOCA fonctionne
- [ ] Sync calculé correctement
- [ ] Smoothing réduit le bruit
- [ ] Dry-run passe